In [ ]:
import datetime

import pandas as pd
import numpy  as np
import os
from pathlib import Path
from common.modules.enums import Exchange, Direction
from statsmodels.tsa.stattools import adfuller
from connector.influxdb.influxdb_wrapper import influx

path_ex = Path(os.getcwd()).parent.parent.joinpath('experiments')

For any series of events (bars) beyond exceeding expectations, eg., vector of ts/sum of runs -
are returns exceeding expections ?
Vector events x Mat(return products x ts) => another Mat. what's that inner product exactly?
    what's a signficantly different return?
H0: p(r > 1 | signficant imbalance) == 0.5
H0: p(r < 1 | signficant imbalance) == 0.5
H1  p(r > 1 | significant imbalance) > 0.5

r needs to be stationary for these tests, such as that E0(mean(r)) == Product(r) == 1. Given an imbalance like that shock, expect product(r) to deviate how much?

1. bars likely not stationary. Make stationary or just use EWM to search for outliers.
2. Get vector of outliers. Time / amplitude
3.

Alpha: Signifacant one-sided volume imbalance without significant price change -> Order Book holds. Precedes price move
Data needed: rolling return, rolling trade volume imbalance.
Feature: rolling volume imbalance per unit of return. if large, means much volum no price change indicative of reversal

need accessory: order book imbalance (ewm across levels, low levels more important than higher levels) - start pulling exchange data in realtime and append to text file split by exchange, symbol, trade/quote, day. Soonish have enough to model with it ~ 1week!

In [ ]:
p_ex = 'ex 2022-02-02 190202'
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2020, 1, 5)

In [ ]:
with open(os.path.join(path_ex, p_ex, 'lala.json'), 'r') as f:
    print(f.read())

- if not stationary, use fractional differentiation to turn it into 1
- get special events, 1 sigma away, 2 sigma away from mean... is not gaussian - so just select outliers... top 5%. get all above expected value ??? min/low 5 % may not be outliers...
-       rather... what;s the expecation? EWM ...
- for each special event analyse state and profit
    state eventually defined by hundreds of stationary series with thousands of events per year
    that's where ML starts...

In [ ]:
df = influx.query(query=f'''
            from(bucket:"trading")
            |> range(start: {start.isoformat()}Z, stop: {end.isoformat()}Z)
            |> filter(fn:(r) => r._measurement == "trade bars" and
                                r.asset == "ethusd" and
                                r._field == "imbalance_size" and
                                r.information == "imbalance_per_plus_tick"
                     )
            '''
            )

In [ ]:
def is_stationary(arr: np.array):
    """
    Augmented Dickey-Fuller test
    https://machinelearningmastery.com/time-series-data-stationary-python/
    p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.
    """
    result = adfuller(df['imbalance_size'].values)
    print(f'''p-value: {result[1]} - ADF Statistic: {result[0]}''')
    # for key, value in result[4].items():
    #     print('\t%s: %.3f' % (key, value))
    return result[1] <= 0.05

In [ ]:
if not is_stationary(df.values):
    print('Fractional differencing on series')
    raise NotImplementedError

In [ ]:
low = df['imbalance_size'].quantile(q=0.025)
high = df['imbalance_size'].quantile(q=1-0.025)

In [ ]:
df['imbalance_size'].sort_values()[99 * len(df)// 100]

In [ ]:
df['imbalance_size'].sort_values()[1 * len(df)// 100]